# OIH Queries 


In [1]:
import rdflib
import gzip

with gzip.open('./oceanexperts_graph.nq.gz', 'rb') as f:
    file_content = f.read()

g = rdflib.Graph()
g.parse(data = file_content, format="nquads")



<Graph identifier=N510198d4b3494a27b29e043f62b2f306 (<class 'rdflib.graph.Graph'>)>

In [2]:
qres = g.query(
    """prefix schema: <https://schema.org/>
    SELECT DISTINCT ?s ?name
       WHERE {
          ?s a schema:Course .
          ?s schema:name ?name
       }
       LIMIT 10""")

for row in qres:
    print("%s Course Name: %s" % row)

N751efae3514c464da129147a0de25df9 Course Name: IOC Training Course and Identification Qualification in Harmful Marine Microalgae including optional workshops on enumeration and culture techniques, 2018
N046d9133e7d249b09cd1bf168f3c5c3f Course Name: Community Model Interface for Tsunami (ComMIT) Training Workshop 
Nfe5ffaca7a1646ad9597fa49e271dbfb Course Name: eSurge Training in Applying EO Data to Storm Surge Modelling and Forecasting
N5a1ae6f5e0dc4d80ba974aa66a4af11b Course Name: OTGA Training Course: Quality Management Framework
Nf7abbc59964d48bcb7f95ce3977457fa Course Name: ECOMAMA GIS Training Course
N604ec3e3961645ac86703d4730c72e8f Course Name: Regional Training:  ICG/PTWS Pacific Tsunami Warning Center (PTWC) New Enhanced Tsunami Products
Na6251964049c47ba98ca014cd68d6914 Course Name: OTGA Training Course: Discovery and Use of Operational Ocean Data Products and Services
N3100fa9e1b28467ea902f108f7973edd Course Name: OTGA/INVEMAR Training Course: Áreas Marinas Protegidas (AMPs)


## Oceans UI test query

In [ ]:
prefix prov: <http://www.w3.org/ns/prov#>
        PREFIX con: <http://www.ontotext.com/connectors/lucene#>
        PREFIX luc: <http://www.ontotext.com/owlim/lucene#>
        PREFIX con-inst: <http://www.ontotext.com/connectors/lucene/instance#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX schema: <https://schema.org/>
        PREFIX schemaold: <http://schema.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

        SELECT DISTINCT ?g  ?s  ?wat ?orgname ?domain ?type ?score ?name ?url ?lit ?description ?headline
        WHERE
        {

           ?lit bds:search "coral" .
           ?lit bds:matchAllTerms "false" .
           ?lit bds:relevance ?score .
           graph ?g {
            ?s ?p ?lit .
            ?s rdf:type ?type . 
            OPTIONAL { ?s schema:name ?name .   }
            OPTIONAL { ?s schema:headline ?headline .   }
            OPTIONAL { ?s schema:url ?url .   }
            OPTIONAL { ?s schema:description ?description .    }
          }
           ?sp prov:generated ?g  .
           ?sp prov:used ?used .
           ?used prov:hadMember ?hm .
           ?hm prov:wasAttributedTo ?wat .
           ?wat rdf:name ?orgname .
           ?wat rdfs:seeAlso ?domain


        }
        ORDER BY DESC(?score)
        LIMIT 30
        OFFSET 0

## AquaDocs Alignment Test Query

In [ ]:
prefix prov: <http://www.w3.org/ns/prov#>
        PREFIX con: <http://www.ontotext.com/connectors/lucene#>
        PREFIX luc: <http://www.ontotext.com/owlim/lucene#>
        PREFIX con-inst: <http://www.ontotext.com/connectors/lucene/instance#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX schema: <https://schema.org/>
        PREFIX schemaold: <http://schema.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

        SELECT DISTINCT ?g  ?s  ?wat ?orgname ?domain ?type ?score ?name ?url ?lit ?description ?headline
        WHERE
        {

           ?lit bds:search "coral reef" .
           ?lit bds:matchAllTerms "false" .
           ?lit bds:relevance ?score .
           ?s ?p ?lit .

           graph ?g {
            ?s ?p ?lit .
            ?s rdf:type ?type . 
            OPTIONAL { ?s schema:name ?name .   }
            OPTIONAL { ?s schema:headline ?headline .   }
            OPTIONAL { ?s schema:url ?url .   }
            OPTIONAL { ?s schema:description ?description .    }
          }
     


        }
        ORDER BY DESC(?score)
        LIMIT 30
        OFFSET 0